# Simulating Images from the MAST telescope

In this project, we use data from the Gaia telescope and create an image which will be captured by the CCD on the MAST telescope. This will be helpful for the tracking and guiding of the telescope.

In [1]:
# Import the required libraries

# astroquery for retrieving data from Gaia
from astroquery.gaia import Gaia

# astropy libraries for calculations
from astropy import units as u
from astropy import wcs
from astropy.constants import h, c
from astropy import convolution

# Data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib ipympl
from scipy import signal

# Manipulating & parsing files
import re
import os
from glob import glob

# Making the notebook interactive
import ipywidgets as widgets
from IPython.display import display

In order to use this notebook, please first run all of the cells. You will get textboxes to fill the desired coordinates, different parameters, etc. Then, you can press the button "Simulate Imgae", that will save the image in the requested path.

In [2]:
# Defining the constants of the notebook

# Constants from files
qe = pd.read_csv('./ccd_data/QE.csv',delimiter=',',index_col=0) # Quantum Efficiency
dark = pd.read_csv('./ccd_data/dark_current.csv',delimiter=',') # Dark Current

# Gaia Flux Density
fd_unit = u.J/u.m**2/u.s/u.nm
d_filters = {'Band':['G','BP','RP'],'Wavelength':[639.07*u.nm,518.26*u.nm,782.51*u.nm],'Bandpass':[454.82*u.nm,265.9*u.nm,292.75*u.nm],'clam':[1.346e-21*fd_unit,3.009e-21*fd_unit,1.638e-21*fd_unit]} 
filters = pd.DataFrame(data=d_filters)

# MAST CCD constants
# Pixels in the CCD
ccd_width_px = 4656
ccd_height_px = 3520

# Buffer of pixels when querying data and performing the convolution
ccd_width_buff_px = 512
ccd_height_buff_px = 512

# The lenght of each pixel in the CCD
ccd_px_side_length_micron = 3.8 * u.micron
ccd_width_micron = ccd_width_px * ccd_px_side_length_micron
ccd_height_micron = ccd_height_px * ccd_px_side_length_micron
ccd_width_buff_micron = ccd_width_buff_px * ccd_px_side_length_micron
ccd_height_buff_micron = ccd_height_buff_px * ccd_px_side_length_micron

# The platescale of MAST
plate_scale_arcsec_micron = 0.11 * u.arcsec/u.micron

# Pixel field of view
px_fov = plate_scale_arcsec_micron * ccd_px_side_length_micron

# CCD field of view
ccd_width_range_arcsec = ccd_width_micron * plate_scale_arcsec_micron
ccd_height_range_arcsec = ccd_height_micron * plate_scale_arcsec_micron
ccd_width_buff_range_arcsec = ccd_width_buff_micron * plate_scale_arcsec_micron
ccd_height_buff_range_arcsec = ccd_height_buff_micron * plate_scale_arcsec_micron

# Converting from arcsec to degrees
ccd_width_range_deg = ccd_width_range_arcsec.to(u.deg)
ccd_height_range_deg = ccd_height_range_arcsec.to(u.deg)
ccd_width_buff_range_deg = ccd_width_buff_range_arcsec.to(u.deg)
ccd_height_buff_range_deg = ccd_height_buff_range_arcsec.to(u.deg)

# MAST collecting area
mast_collecting_area_m2 = 0.09 * np.pi * u.m**2

# Saturation of the CCD per pixel
ccd_full_well = 20000 * u.electron

# CCD where light is captured
ccd_live_right_deg = 0.18 * u.deg
ccd_live_left_deg = 0.146 * u.deg

# CCD where light is not captured (CCD under the optical fiber)
ccd_dead_left_px = int(np.ceil(ccd_live_left_deg / px_fov.to(u.deg)))
ccd_dead_right_px = int(ccd_width_px - np.floor(ccd_live_right_deg / px_fov.to(u.deg)) - 1)

# Gaia constants
gaia_collecting_area_m2 = 0.7 * u.m**2

# Gaia-MAST constants
gaia_mast_attenuation = 0.5

In [3]:
# Defining the widgets for user input

ra_widget = widgets.BoundedFloatText(
    value=279.3643447,
    min=0,
    max=360,
    step=0.001,
    description='RA:'
)

dec_widget = widgets.BoundedFloatText(
    value=38.79739118,
    min=-90,
    max=90,
    step=0.001,
    description='DEC:'
)

max_mag_widget = widgets.FloatText(
    value=15,
    step=0.01,
    description='Max Mag:'
)

t_exp_widget = widgets.BoundedFloatText(
    value=1,
    step=0.1,
    min=0.1,
    description='Exp Time:'
)

seeing_arcsec_widget = widgets.BoundedFloatText(
    value=1.5,
    step=0.1,
    min=0,
    description='Seeing:'
)

bdg_mag_to_arcsec_squared_widget = widgets.BoundedFloatText(
    value=20.5,
    step=0.1,
    min=0,
    description='Background:'
)

temperature_widget = widgets.Dropdown(
    options=dark['temp'],
    value=dark['temp'].iloc[0],
    description='Op Temp:'
)

In [4]:
# General function to query from Gaia using ADQL
# Help can be found be found at: https://www.cosmos.esa.int/web/gaia-users/archive/writing-queries

def query_gaia(query):
    job = Gaia.launch_job(query)
    return job.get_results()

In [5]:
# Defining the query to get sources captured by Gaia in a specific region
# Documentation about the table GAIADR3.GAIA_SOURCE can be found at: https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html

def query_gaia_for_sources_in_region(max_mag, ra_deg, dec_deg, ccd_width_range_deg, ccd_height_range_deg):
    query = '''SELECT SOURCE_ID, RA, DEC, PHOT_G_MEAN_MAG, HAS_XP_SAMPLED,
    PHOT_G_MEAN_FLUX AS PHOT_G_MEAN_FLUX_GAIA, PHOT_G_MEAN_FLUX_ERROR AS PHOT_G_MEAN_FLUX_ERROR_GAIA,
    PHOT_BP_MEAN_FLUX AS PHOT_BP_MEAN_FLUX_GAIA, PHOT_BP_MEAN_FLUX_ERROR AS PHOT_BP_MEAN_FLUX_ERROR_GAIA,
    PHOT_RP_MEAN_FLUX AS PHOT_RP_MEAN_FLUX_GAIA, PHOT_RP_MEAN_FLUX_ERROR AS PHOT_RP_MEAN_FLUX_ERROR_GAIA
    FROM gaiadr3.gaia_source
    WHERE (ra BETWEEN {ra_min} AND {ra_max})
    AND (dec BETWEEN {dec_min} AND {dec_max})
    AND phot_g_mean_mag < {max_mag}
    ORDER BY PHOT_G_MEAN_MAG
    '''.format(
        max_mag=max_mag,
        ra_min=ra_deg-(ccd_width_range_deg.value/2), ra_max=ra_deg+(ccd_width_range_deg.value/2),
        dec_min=dec_deg-(ccd_height_range_deg.value/2), dec_max=dec_deg+(ccd_height_range_deg.value/2)
    )
    return query_gaia(query)

In [6]:
# Input the flux as seen by Gaia (in electrons/sec as given by GAIA_SOURCE)
# Output the flux density of the source
# The conversion is done according to the documentation: https://gea.esac.esa.int/archive/documentation/GDR3/Data_processing/chap_cu5pho/cu5pho_sec_photProc/cu5pho_ssec_photCal.html#SSS3.P2

def calc_gaia_flux_density_from_gaia_flux(sources_table):
    for filter in d_filters['Band']:
        sources_table['flux_' + filter + '_density_gaia'] = filters[filters['Band'] == filter]['clam'].iloc[0] * sources_table['phot_' + str.lower(filter) + '_mean_flux_gaia'].value
        sources_table['flux_' + filter + '_density_error_gaia'] =  filters[filters['Band'] == filter]['clam'].iloc[0] * sources_table['phot_' + str.lower(filter) + '_mean_flux_error_gaia'].value

In [7]:
# Calculate the mean quantum efficiency of Gaia, based on the wavelenghts of intrest

def calculate_effective_qe(min_wavelen, max_wavelen):
    return np.mean(qe[qe['Wavelength'].isin(qe['Wavelength'].where((qe['Wavelength'] >= min_wavelen) & (qe['Wavelength'] < max_wavelen)).dropna())]['QE'])

In [8]:
# Given the flux density as seen at Gaia, calculate the flux as measured in MAST

def convert_gaia_flux_density_to_mast_flux(sources_table):
    for filter in d_filters['Band']:
        lam0 = filters[filters.Band == filter].iloc[0]['Wavelength']
        bandpass = filters[filters.Band == filter].iloc[0]['Bandpass']
        flux_density_inc_mast =  gaia_mast_attenuation * sources_table['flux_' + filter + '_density_gaia']
        flux_meas_mast = calculate_effective_qe(lam0-bandpass/2,lam0+bandpass/2)* flux_density_inc_mast * bandpass
        power_meas_mast = flux_meas_mast * mast_collecting_area_m2
        sources_table['phot_' + filter + '_flux_mast'] = (power_meas_mast * lam0.to(u.m) / (h * c)) * u.electron 

        flux_density_inc_error_mast =  gaia_mast_attenuation * sources_table['flux_' + filter + '_density_error_gaia']
        flux_meas_error_mast = calculate_effective_qe(lam0-bandpass/2,lam0+bandpass/2) * flux_density_inc_error_mast * bandpass
        power_meas_error_mast = flux_meas_error_mast * mast_collecting_area_m2
        sources_table['phot_' + filter + '_flux_rel_error_mast'] = np.sqrt(((power_meas_error_mast * lam0.to(u.m) / (h * c)).value)**2 + ((power_meas_mast * bandpass.to(u.m) / (2 * h * c)).value)**2) / sources_table['phot_' + filter + '_flux_mast'].value * 100 * u.percent

In [9]:
# We assume the source is a point-like source. Since it doesn't fall at an exact location, we need to split it between the four pixels in the area.
# w_px, h_px are floats of the exact location of the point-like source
# width, height are the dimensions of the simulated image

def four_px(w_px,h_px,width,height):
    lst = []
    if (w_px <0 or w_px >= width or h_px < 0 or h_px >= height):
        return lst
    wf = np.floor(w_px)
    wc = np.ceil(w_px)
    hf = np.floor(h_px)
    hc = np.ceil(h_px)
    if np.allclose(w_px,wf): 
        w_floor_res = 1
        w_ceil_res = 0
    else: 
        w_floor_res = np.round(w_px - wf,3)
        w_ceil_res = np.round(wc - w_px,3)
    if np.allclose(h_px,hf): 
        h_floor_res = 1
        h_ceil_res = 0
    else: 
        h_floor_res = np.round(h_px - hf,3)
        h_ceil_res = np.round(hc - h_px,3)    

    if (wf >= 0 and hf >= 0):
        lst.append([wf ,hf ,w_floor_res*h_floor_res])
    if (wc < width and hf >= 0):
        lst.append([wc ,hf ,w_ceil_res*h_floor_res])
    if (wf >= 0 and hc < height):
        lst.append([wf ,hc ,w_floor_res*h_ceil_res])
    if (wc < width and hc < height):
        lst.append([wc , hc ,w_ceil_res*h_ceil_res])
    return lst

In [10]:
# Add the sources to the base image
# image - Matrix for the image
# sources_table - Sources found by Gaia in that region
# wcs_dict - astropy.wcs object to convert from coordinates to pixels
# t_exp - Image exposure time
def add_sources(image, sources_table, wcs_dict, t_exp):
    pixs = wcs_dict.wcs_world2pix(sources_table['ra'].value, sources_table['dec'].value, 0)
    flux = np.round(sources_table['phot_G_flux_mast'].value)
    brightness = np.random.poisson(lam = flux) * t_exp / u.s 
    for i in range(len(pixs[0])):
        lst = four_px(pixs[0][i],pixs[1][i],image.shape[1],image.shape[0])
        for ls in lst:
            image[int(ls[1]),int(ls[0])] += ls[2] * brightness[i]
    return image

In [11]:
# Add the seeing effect to the image caused by the atmosphere

def add_seeing(image, seeing_arcsec):
    seeing_px = seeing_arcsec / plate_scale_arcsec_micron / ccd_px_side_length_micron
    sigma = seeing_px/2.355 # fhwm = 2*sqrt(2*ln(2))*sigma
    ker = convolution.Gaussian2DKernel(x_stddev = sigma , y_stddev = sigma)
    return convolution.convolve(image,ker)

In [12]:
# Add background noise to the image

def add_bgd_noise(image, mean_bdg_mag_to_arcsec_squared, t_exp):
    rate = 2e5*u.electron/u.s/u.arcsec**2*10**(-0.4*(mean_bdg_mag_to_arcsec_squared-10))*(ccd_px_side_length_micron*plate_scale_arcsec_micron)**2
    mean = (rate * t_exp).value
    image += np.random.normal(loc = mean, scale = np.sqrt(mean), size = (image.shape[0],image.shape[1])) # for now, brightness = number of photo electrons 
    return image

In [13]:
# Remove the dead regions, where the optical fiber sits, from the image

def remove_dead(image):
    image[ccd_dead_left_px:ccd_dead_right_px] = 0
    return image

In [14]:
# Add the dark noise to the image

def add_read_dark_noise(image, temperature, read_rms, t_exp):
    dark_rate = dark[dark.temp == temperature].iloc[0]['e/s/pix'] * u.electron/u.s
    mean = (dark_rate * t_exp).value
    var = (dark_rate * t_exp).value + read_rms.value**2
    image += np.random.normal(loc = mean, scale = np.sqrt(var), size = (image.shape[0],image.shape[1]))
    return image

In [15]:
# Perform a convolution on a photo, where each region in the photo will be convolved with a different kernel.
# img - the image to go under the convolution
# kernel_folder - folder with .npy files describing the kernels in the different regions. The region on which the kernel will act on is described by the file name "{row_lower}:{row_upper}X{column_lower}:{column_upper}" (indices are inclusive) and is relative to the next parameters
# img_{row/col}_lower_lim - which pixel will be mapped to (0,0) after the convolution
# conv_img_{row/col} - the dimension of the image after the convolution

def convolve_region_based_kernel(img, kernel_folder, img_row_lower_lim, img_col_lower_lim, conv_img_row, conv_img_col):
    img_conv = np.zeros((conv_img_row, conv_img_col))
    for kernel_file_path in glob(os.path.join(kernel_folder, '*')):
        kernel = np.load(kernel_file_path)
        k_rows, k_cols = kernel.shape
        px_match = re.search('(\d+):(\d+)X(\d+):(\d+)', kernel_file_path)
        patch_rel_row_start = int(px_match.group(1))
        patch_rel_row_end = int(px_match.group(2))
        patch_rel_col_start = int(px_match.group(3))
        patch_rel_col_end = int(px_match.group(4))
        patch_row_start = img_row_lower_lim+patch_rel_row_start-k_rows+1
        patch_row_end = img_row_lower_lim+patch_rel_row_end+1
        patch_col_start = img_col_lower_lim+patch_rel_col_start-k_cols+1
        patch_col_end = img_col_lower_lim+patch_rel_col_end+1
        patch = img[patch_row_start:patch_row_end,patch_col_start:patch_col_end]
        patch_conv = signal.convolve2d(patch, kernel, mode='valid')
        img_conv[patch_rel_row_start:patch_rel_row_end+1,patch_rel_col_start:patch_rel_col_end+1] = patch_conv
    return img_conv

In [16]:
# Create the final sources table from Gaia (from querying Gaia to getting the correct measurment for MAST)

def create_sources_table(max_mag, ra_center, dec_center):
    coor_width_range_deg = (ccd_width_range_deg + ccd_width_buff_range_deg) / np.cos(np.deg2rad(dec_center))
    coor_height_range_deg = ccd_height_range_deg + ccd_height_buff_range_deg
    sources_table = query_gaia_for_sources_in_region(max_mag, ra_center, dec_center, coor_width_range_deg, coor_height_range_deg)
    calc_gaia_flux_density_from_gaia_flux(sources_table)
    convert_gaia_flux_density_to_mast_flux(sources_table)
    return sources_table

In [17]:
def create_wcs(ra_center, dec_center):
    wcs_input_dict = {
    'CTYPE1': 'RA---TAN', 
    'CUNIT1': 'deg', 
    'CDELT1': (px_fov.to(u.deg)).value, 
    'CRPIX1': (ccd_width_px+ccd_width_buff_px)/2 - 1, 
    'CRVAL1': ra_center, 
    'NAXIS1': ccd_width_px+ccd_width_buff_px,
    'CTYPE2': 'DEC--TAN', 
    'CUNIT2': 'deg', 
    'CDELT2': (px_fov.to(u.deg)).value, 
    'CRPIX2': (ccd_height_px+ccd_height_buff_px)/2 - 1, 
    'CRVAL2': dec_center, 
    'NAXIS2': ccd_height_px+ccd_height_buff_px
    }
    return wcs.WCS(wcs_input_dict)

In [18]:
# Create the simulated image from MAST based on the sources table prepered previously

def create_image(sources_table, ra_center , dec_center, t_exp, bdg_mean_mag_to_arcsec_squared, seeing_arcsec, temperature):
    wcs_dict = create_wcs(ra_center, dec_center)
    image = np.zeros((ccd_height_px + ccd_height_buff_px, ccd_width_px + ccd_width_buff_px))
    image = add_sources(image, sources_table, wcs_dict, t_exp)
    image = add_bgd_noise(image, bdg_mean_mag_to_arcsec_squared, t_exp)
    image = add_seeing(image,seeing_arcsec)
    image = convolve_region_based_kernel(image, r'./kernels', int(ccd_height_buff_px/2), int(ccd_width_buff_px/2), ccd_height_px, ccd_width_px)
    image = remove_dead(image)
    image = add_read_dark_noise(image,temperature,3.6*u.electron,t_exp)
    return image

In [19]:
# Plot the image in the notebook

def plot_image(image):
    fig,ax = plt.subplots(figsize=(8,8),dpi=120)
    mappable = ax.imshow(image, origin='lower', cmap='Greys', norm=colors.LogNorm())
    ax.set_xlabel('Horizontal Axis (px)', fontdict={'size': 12})
    ax.set_ylabel('Vertical Axis (px)', fontdict={'size': 12})
    cbar = fig.colorbar(mappable, ax=ax, shrink=0.7, pad = 0.05, format='%1.1E')
    cbar.set_label('Electron Count', labelpad=10, rotation = 270, fontdict={'size':12})
    plt.show()

In [20]:
# Simulate the image based on the telescope parameters and plot it to the user

def simulate_image(max_mag, ra, dec, t_exp, bdg_mean_mag_to_arcsec_squared, seeing_arcsec, temperature):
    sources_table = create_sources_table(max_mag, ra, dec)
    image = create_image(sources_table, ra, dec, t_exp, bdg_mean_mag_to_arcsec_squared, seeing_arcsec, temperature)
    plot_image(image)

In [21]:
# Display the widgets in the notebook

display(ra_widget)
display(dec_widget)
display(max_mag_widget)
display(t_exp_widget)
display(seeing_arcsec_widget)
display(bdg_mag_to_arcsec_squared_widget)
display(temperature_widget)

BoundedFloatText(value=279.3643447, description='RA:', max=360.0, step=0.001)

BoundedFloatText(value=38.79739118, description='DEC:', max=90.0, min=-90.0, step=0.001)

FloatText(value=15.0, description='Max Mag:', step=0.01)

BoundedFloatText(value=1.0, description='Exp Time:', min=0.1, step=0.1)

BoundedFloatText(value=1.5, description='Seeing:', step=0.1)

BoundedFloatText(value=20.5, description='Background:', step=0.1)

Dropdown(description='Op Temp:', options=(-20, -15, -10, -5, 0, 5, 10, 15, 25), value=-20)

In [22]:
# Create the button to start the simulation

button_simulate_image_widget = widgets.Button(description='Simulate Image')

# Create the output for the image

output_widget = widgets.Output()

def on_button_click(button):
    with output_widget:
        simulate_image(max_mag_widget.value,
                       ra_widget.value,
                       dec_widget.value,
                       t_exp_widget.value,
                       bdg_mag_to_arcsec_squared_widget.value,
                       seeing_arcsec_widget.value,
                       temperature_widget.value)

button_simulate_image_widget.on_click(on_button_click)

In [23]:
display(button_simulate_image_widget)

Button(description='Simulate Image', style=ButtonStyle())

In [24]:
display(output_widget)

Output()